In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
tf.enable_eager_execution()


Using TensorFlow backend.


In [2]:
%%bash
pip install keras==2.2.4

In [3]:
def sqr(var1):
    return tf.math.square(var1)

In [4]:
alpha = tf.Variable(1.0)
beta = tf.Variable(1.0)
phi = tf.Variable(1.0)
theta = tf.Variable(1.0)
gamma = tf.Variable(1.0)
r = tf.Variable(1)
t = tf.Variable(1)
rl = tf.Variable(1)

In [5]:
c1 = tf.Variable(1/3)
c2 = tf.Variable(2/3)

In [6]:
sin_alpha = tf.math.sin(alpha)
sin_two_alpha = tf.math.sin(2*alpha)
sin_beta = tf.math.sin(beta)
sin_two_beta = tf.math.sin(2*beta)
sin_phi = tf.math.sin(phi)
sin_two_phi = tf.math.sin(2*phi)
sin_theta = tf.math.sin(theta)
sin_two_theta = tf.math.sin(2*theta)
sin_gamma = tf.math.sin(gamma)
sin_phi_beta_subt = tf.math.sin(phi - beta)


cos_alpha = tf.math.cos(alpha)
cos_beta = tf.math.cos(beta)
cos_two_beta = tf.math.cos(2*beta)
cos_phi = tf.math.cos(phi)
cos_two_phi = tf.math.cos(2*phi)
cos_theta = tf.math.cos(theta)
cos_gamma = tf.math.cos(gamma)
cos_beta_phi_subt = tf.math.cos(beta - phi)
cos_beta_theta_add = tf.math.cos(beta + theta)

cosec_theta = tf.math.acos(theta)
cot_theta = tf.math.atan(theta)
cot_alpha = tf.math.atan(alpha)


In [7]:
Z1 = sqr(sin_alpha)*sqr(sin_theta)*sqr(cos_beta_phi_subt) + sin_two_theta*sin_alpha*cos_beta_phi_subt - c1

Z2 = sqr(sin_alpha)*sqr(cos_theta)*sqr(cos_beta_phi_subt) - sin_alpha*cos_alpha*sin_two_theta*cos_beta_phi_subt + sqr(cos_alpha)*sqr(sin_theta) - c1

Z3 = sqr(sin_alpha)*sqr(cosec_theta)*sqr(sin_phi_beta_subt) - c1

Z4 = sin_two_theta

Z5 = sqr(sin_alpha)*sin_two_phi + sqr(cos_alpha)*sin_two_theta + sin_two_beta*sqr(sin_alpha)*cos_two_phi + sin_two_alpha*cot_theta*(cos_beta*sin_phi + sin_beta*cos_phi)

Z6 = sqr(sin_alpha)*cot_theta*(cos_beta*sin_beta*cos_two_phi - sqr(cos_beta)*sin_two_phi) + sqr(sin_alpha)*sin_phi_beta_subt

Z7 = sqr(sin_alpha)*(sqr(cos_theta)*sqr(cos_beta_theta_add) + sqr(cos_beta*sin_phi + sin_beta) -sqr(sin_beta)*sqr(sin_phi)  + sqr(cos_alpha)*sqr(sin_theta) - c2  )

Z8 = sqr(cos_beta)*sqr(sin_alpha)

Z9 = sqr(sin_alpha) * ( sin_beta*cos_beta*(sqr(cosec_theta)*cos_two_phi + cosec_theta*cot_theta + cos_two_phi) + sin_beta*cot_alpha*cot_theta*cos_phi - sin_phi*cos_phi*cos_two_beta*cot_theta*(cot_theta - cosec_theta))

In [8]:
neural_net = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(8,)),
    tf.keras.layers.Dense(124, activation = tf.nn.relu),
    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)
])


In [9]:
def u(t, r, alpha, beta, theta, phi, gamma, rl):
    u = neural_net([t, r, alpha, beta, theta, phi, gamma, rl], weights, biases)
    return u    


In [10]:
def f(u):
    u_t = tf.gradients(u, t)[0]
    u_r = tf.gradients(u, r)[0]
    u_rr = tf.gradients(u_r, r)[0]
    u_sq_rr = tf.gradients(sqr(r)*u_r, r)[0]
    
    u_th = tf.gradients(u, theta)[0]
    u_th = tf.gradients(u_th, theta)[0]
    u_th_sin = tf.gradients(tf.math.sin(theta)*u_th, theta)[0]
    u_th_r = tf.gradients(u_th, r)[0]
    u_th_ph = tf.gradients(u_th, phi)[0]
    
    u_ph = tf.gradients(u, phi)[0]
    u_phph = tf.gradients(u_ph, phi)[0]
    u_ph_r = tf.gradients(u_ph, r)[0]
    
    term1 = (1/sqr(r))*(u_sq_rr)
    term2 = (1/sqr(r*sin_theta))*u_th_sin
    term3 = (1/sqr(r*sin_theta))*u_phph
    del_sqr_spherical = term1 + term2 + term3
    
    Z = Z1*u_rr + (1/sqr(r))*(Z2*u_thth + Z3*u_phph + Z4*u_th_r - Z5*u_ph_r - Z6*u_th_ph + Z8*u_th + Z9*u_phi)+(1/r)*Z7*u_r
    f = u_t - del_sqr_spherical - Z               
    return f               

In [11]:
def loss(u):
    u_t = tf.gradients(u, t)[0]
    u_r = tf.gradients(u, r)[0]
    u_rr = tf.gradients(u_r, r)[0]
    u_sq_rr = tf.gradients(sqr(r)*u_r, r)[0]
    
    u_th = tf.gradients(u, theta)[0]
    u_th = tf.gradients(u_th, theta)[0]
    u_th_sin = tf.gradients(tf.math.sin(theta)*u_th, theta)[0]
    u_th_r = tf.gradients(u_th, r)[0]
    u_th_ph = tf.gradients(u_th, phi)[0]
    
    u_ph = tf.gradients(u, phi)[0]
    u_phph = tf.gradients(u_ph, phi)[0]
    u_ph_r = tf.gradients(u_ph, r)[0]
    
    term1 = (1/sqr(r))*(u_sq_rr)
    term2 = (1/sqr(r*sin_theta))*u_th_sin
    term3 = (1/sqr(r*sin_theta))*u_phph
    del_sqr_spherical = term1 + term2 + term3
    
    Z = Z1*u_rr + (1/sqr(r))*(Z2*u_thth + Z3*u_phph + Z4*u_th_r - Z5*u_ph_r - Z6*u_th_ph + Z8*u_th + Z9*u_phi)+(1/r)*Z7*u_r
    f = u_t - del_sqr_spherical - Z               
    return f               

In [12]:
class model:
    
    def __init__(self):
        xavier=tf.keras.initializers.GlorotUniform()
        self.l1=tf.keras.layers.Dense(64,kernel_initializer=xavier,activation=tf.nn.relu,input_shape=[1])
        self.l2=tf.keras.layers.Dense(64,kernel_initializer=xavier,activation=tf.nn.relu)
        self.out=tf.keras.layers.Dense(1,kernel_initializer=xavier)
        self.train_op = tf.keras.optimizers.Adagrad(learning_rate=0.1)
        
    # Running the model
    def run(self,X):
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        return boom2
      
    #Custom loss fucntion
    def get_loss(self,X,Y):
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        return f(boom2)
      
    # get gradients
    def get_grad(self,X,Y):
        with tf.GradientTape() as tape:
            tape.watch(self.l1.variables)
            tape.watch(self.l2.variables)
            tape.watch(self.out.variables)
            L = self.get_loss(X,Y)
            g = tape.gradient(L, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]])
        return g
      
    # perform gradient descent
    def network_learn(self,X,Y):
        g = self.get_grad(X,Y)
        self.train_op.apply_gradients(zip(g, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]]))



In [13]:
input_shape=(8,1)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(8, activation=tf.nn.sigmoid, input_shape=input_shape),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.sigmoid),
  tf.keras.layers.Dense(1)
])


In [14]:
inp = tf.Variable(np.random.normal(size=(1,8,1)), dtype=tf.float32)

with tf.GradientTape() as tape:
    preds = model(inp)

grads = tape.gradient(preds, inp)

In [87]:
inputs = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])
#inputs = tf.reshape(inputs,[1,8,1])
#inputs = tf.Variable(inputs)
#inputs = tf.cast(inputs,tf.float32)
t = tf.Variable(inputs[0],dtype=tf.float32)
r = tf.Variable(inputs[1],dtype=tf.float32)
alpha= tf.Variable(inputs[2],dtype=tf.float32)
beta= tf.Variable(inputs[3],dtype=tf.float32)
theta= tf.Variable(inputs[4],dtype=tf.float32)
phi= tf.Variable(inputs[5],dtype=tf.float32)
gamma= tf.Variable(inputs[6],dtype=tf.float32)
rl= tf.Variable(inputs[7],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape3:
    tape3.watch(model.trainable_variables)
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape1:
            inp = tf.reshape(tf.stack([t, r, alpha, beta, theta, phi, gamma, rl]),[1,8,1])
            #inp = tf.cast(inp,tf.float32)
            tape1.watch([t, r, alpha, beta, theta, phi, gamma, rl])
            preds = model(inp)
        d_t,d_r,d_al,d_bt,d_th,d_ph,d_gm,d_rl = tape1.gradient(preds, [t, r, alpha, beta, theta, phi, gamma, rl])
    d_r_r = tape2.gradient(d_r, r)
    d_th_r = tape2.gradient(d_th, r)
    d_th_ph = tape2.gradient(d_th, phi)
    d_th_th = tape2.gradient(d_th, theta)
    d_ph_ph = tape2.gradient(d_ph, phi)
    d_ph_r = tape2.gradient(d_ph, r)
    d_sq_r_r = 2*r*d_r + sqr(r)*d_r_r
    d_th_sin = tf.math.cos(theta)*d_th + tf.math.sin(theta)*d_th_th
    term1 = tf.math.reciprocal(sqr(r))*(d_sq_r_r)
    term2 = tf.math.reciprocal(sqr(tf.math.multiply(r,sin_theta)))*d_th_sin
    term3 = tf.math.reciprocal(sqr(r*sin_theta))*d_ph_ph
    del_sqr_spherical = term1 + term2 + term3
    Z = Z1*d_r_r + (1/sqr(r))*(Z2*d_th_th + Z3*d_ph_ph + Z4*d_th_r - Z5*d_ph_r - Z6*d_th_ph + Z8*d_th + Z9*d_ph)+(1/r)*Z7*d_r
    loss_value = d_t - del_sqr_spherical - Z
gradients = tape3.gradient(loss_value, model.trainable_variables)

print (gradients)


[<tf.Tensor: id=43177, shape=(1, 8), dtype=float32, numpy=
array([[ 0.02722517, -0.00982246, -0.04576404, -0.00952901,  0.04406957,
        -0.00621419,  0.02633443, -0.0374439 ]], dtype=float32)>, <tf.Tensor: id=43164, shape=(8,), dtype=float32, numpy=
array([ 5.0632406e-02, -5.2465528e-04,  3.5308381e-03,  7.4207375e-05,
        6.8550587e-02,  9.7772628e-03, -1.3603499e-02, -5.0524540e-02],
      dtype=float32)>, <tf.Tensor: id=43161, shape=(8, 10), dtype=float32, numpy=
array([[-0.03279346, -0.03089219,  0.05149449,  0.01164417, -0.00881762,
         0.00647309,  0.03977953, -0.03251634, -0.02214342, -0.01230406],
       [ 0.03564646,  0.02820911, -0.04269607, -0.00797464,  0.02704629,
        -0.00620163, -0.0399364 ,  0.03891172,  0.01824311,  0.01068113],
       [ 0.03013376,  0.02345446, -0.0351072 , -0.00639329,  0.02417068,
        -0.00518184, -0.03352305,  0.03316509,  0.0149912 ,  0.00883136],
       [ 0.0104906 ,  0.00612137, -0.00824113, -0.00084682,  0.01296386,
       

In [83]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [18]:
num_samples = 1000
x_size = [1000,8]

In [19]:
x_size

[1000, 8]

In [20]:
points = tf.random.uniform(shape=x_size, dtype=tf.dtypes.float32, seed=10) 

In [21]:
labels = tf.random.uniform(shape=[num_samples], dtype=tf.dtypes.float32, seed=10) 

In [22]:
train_dataset = tuple(zip(points, labels))

In [88]:
def grad(model, inputs, targets):
    t = tf.Variable(inputs[0])
    r = tf.Variable(inputs[1])
    alpha= tf.Variable(inputs[2])
    beta= tf.Variable(inputs[3])
    theta= tf.Variable(inputs[4])
    phi= tf.Variable(inputs[5])
    gamma= tf.Variable(inputs[6])
    rl= tf.Variable(inputs[7])
 
    with tf.GradientTape(persistent=True) as tape3:
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(model.trainable_variables)
            with tf.GradientTape(persistent=True) as tape1:
                inp = tf.reshape(tf.stack([t, r, alpha, beta, theta, phi, gamma, rl]),[1,8,1])
                inp = tf.cast(inp,tf.float32)
                tape1.watch([t, r, alpha, beta, theta, phi, gamma, rl])
                preds = model(inp)
            d_t,d_r,d_al,d_bt,d_th,d_ph,d_gm,d_rl = tape1.gradient(tf.convert_to_tensor(preds), [t, r, alpha, beta, theta, phi, gamma, rl])
        d_r_r = tape2.gradient(d_r, r)
        #d_sq_r_r = tape2.gradient(sqr(r)*d_r, r)
        #d_th_sin = tape2.gradient(tf.math.sin(theta)*d_th, theta)
        d_th_r = tape2.gradient(d_th, r)
        d_th_ph = tape2.gradient(d_th, phi)
        d_th_th = tape2.gradient(d_th, theta)
        d_ph_ph = tape2.gradient(d_ph, phi)
        d_ph_r = tape2.gradient(d_ph, r)
        d_sq_r_r = 2*r*d_r + sqr(r)*d_r_r
        d_th_sin = tf.math.cos(theta)*d_th + tf.math.sin(theta)*d_th_th
        term1 = (1/sqr(r))*(d_sq_r_r)
        term2 = (1/sqr(r*sin_theta))*d_th_sin
        term3 = (1/sqr(r*sin_theta))*d_ph_ph
        del_sqr_spherical = term1 + term2 + term3
        Z = Z1*d_r_r + (1/sqr(r))*(Z2*d_th_th + Z3*d_ph_ph + Z4*d_th_r - Z5*d_ph_r - Z6*d_th_ph + Z8*d_th + Z9*d_ph)+(1/r)*Z7*d_r
        loss_value = d_t - del_sqr_spherical - Z
    gradients = tape3.gradient(loss_value, model.trainable_variables)
    return loss_value,gradients
            
            
    



In [95]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 50

for epoch in range(num_epochs):
      epoch_loss_avg = tf.keras.metrics.Mean()
      epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


  # Training loop - using batches of 32
      for x, y in train_dataset:
        # Optimize the model1) 
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Track progress
        epoch_loss_avg(loss_value)
        print (epoch_loss_avg.result())
        
        # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        #epoch_accuracy(y, model(x, training=True))
        
        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        if epoch % 50 == 0:
            print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(),epoch_accuracy))


tf.Tensor(-23.559175, shape=(), dtype=float32)
tf.Tensor(-13.811335, shape=(), dtype=float32)
tf.Tensor(-9.530814, shape=(), dtype=float32)
tf.Tensor(-7.743513, shape=(), dtype=float32)
tf.Tensor(-6.954638, shape=(), dtype=float32)
tf.Tensor(-6.389032, shape=(), dtype=float32)
tf.Tensor(-5.948779, shape=(), dtype=float32)
tf.Tensor(-5.316556, shape=(), dtype=float32)
tf.Tensor(-4.8526993, shape=(), dtype=float32)
tf.Tensor(-4.862169, shape=(), dtype=float32)
tf.Tensor(-7.782326, shape=(), dtype=float32)
tf.Tensor(-7.3125663, shape=(), dtype=float32)
tf.Tensor(-6.887649, shape=(), dtype=float32)
tf.Tensor(-10.370163, shape=(), dtype=float32)
tf.Tensor(-9.819172, shape=(), dtype=float32)
tf.Tensor(-10.047301, shape=(), dtype=float32)
tf.Tensor(-148.94519, shape=(), dtype=float32)
tf.Tensor(-202.93225, shape=(), dtype=float32)
tf.Tensor(-192.19121, shape=(), dtype=float32)
tf.Tensor(-182.5664, shape=(), dtype=float32)
tf.Tensor(-173.8614, shape=(), dtype=float32)
tf.Tensor(-165.80096, sha

tf.Tensor(-56.766094, shape=(), dtype=float32)
tf.Tensor(-55.770813, shape=(), dtype=float32)
tf.Tensor(-54.80926, shape=(), dtype=float32)
tf.Tensor(-53.88023, shape=(), dtype=float32)
tf.Tensor(-52.982162, shape=(), dtype=float32)
tf.Tensor(-52.108593, shape=(), dtype=float32)
tf.Tensor(-51.268074, shape=(), dtype=float32)
tf.Tensor(-50.441677, shape=(), dtype=float32)
tf.Tensor(-49.653324, shape=(), dtype=float32)
tf.Tensor(-48.889374, shape=(), dtype=float32)
tf.Tensor(-48.14863, shape=(), dtype=float32)
tf.Tensor(-47.42491, shape=(), dtype=float32)
tf.Tensor(-46.727337, shape=(), dtype=float32)
tf.Tensor(-46.04991, shape=(), dtype=float32)
tf.Tensor(-45.392025, shape=(), dtype=float32)
tf.Tensor(-44.752758, shape=(), dtype=float32)
tf.Tensor(-44.13069, shape=(), dtype=float32)
tf.Tensor(-43.526123, shape=(), dtype=float32)
tf.Tensor(-42.937576, shape=(), dtype=float32)
tf.Tensor(-42.365032, shape=(), dtype=float32)
tf.Tensor(-41.807693, shape=(), dtype=float32)
tf.Tensor(-41.26467

tf.Tensor(-26.89584, shape=(), dtype=float32)
tf.Tensor(-26.658138, shape=(), dtype=float32)
tf.Tensor(-26.426567, shape=(), dtype=float32)
tf.Tensor(-26.197987, shape=(), dtype=float32)
tf.Tensor(-25.972229, shape=(), dtype=float32)
tf.Tensor(-25.75098, shape=(), dtype=float32)
tf.Tensor(-25.288033, shape=(), dtype=float32)
tf.Tensor(-25.075546, shape=(), dtype=float32)
tf.Tensor(-24.866589, shape=(), dtype=float32)
tf.Tensor(-24.661226, shape=(), dtype=float32)
tf.Tensor(-24.459196, shape=(), dtype=float32)
tf.Tensor(-24.260391, shape=(), dtype=float32)
tf.Tensor(-24.064875, shape=(), dtype=float32)
tf.Tensor(-23.87239, shape=(), dtype=float32)
tf.Tensor(-23.682936, shape=(), dtype=float32)
tf.Tensor(-23.496468, shape=(), dtype=float32)
tf.Tensor(-23.312899, shape=(), dtype=float32)
tf.Tensor(-23.132208, shape=(), dtype=float32)
tf.Tensor(-22.954273, shape=(), dtype=float32)
tf.Tensor(-22.779062, shape=(), dtype=float32)
tf.Tensor(-22.606483, shape=(), dtype=float32)
tf.Tensor(-22.43

tf.Tensor(-17.773142, shape=(), dtype=float32)
tf.Tensor(-17.667995, shape=(), dtype=float32)
tf.Tensor(-17.564096, shape=(), dtype=float32)
tf.Tensor(-17.461393, shape=(), dtype=float32)
tf.Tensor(-17.359888, shape=(), dtype=float32)
tf.Tensor(-17.261208, shape=(), dtype=float32)
tf.Tensor(-17.162062, shape=(), dtype=float32)
tf.Tensor(-17.064026, shape=(), dtype=float32)
tf.Tensor(-16.967094, shape=(), dtype=float32)
tf.Tensor(-16.871504, shape=(), dtype=float32)
tf.Tensor(-16.776749, shape=(), dtype=float32)
tf.Tensor(-16.683947, shape=(), dtype=float32)
tf.Tensor(-16.59125, shape=(), dtype=float32)
tf.Tensor(-16.500572, shape=(), dtype=float32)
tf.Tensor(-16.409964, shape=(), dtype=float32)
tf.Tensor(-16.330528, shape=(), dtype=float32)
tf.Tensor(-16.24189, shape=(), dtype=float32)
tf.Tensor(-16.154076, shape=(), dtype=float32)
tf.Tensor(-16.067774, shape=(), dtype=float32)
tf.Tensor(-15.982022, shape=(), dtype=float32)
tf.Tensor(-15.897078, shape=(), dtype=float32)
tf.Tensor(-15.8

tf.Tensor(-13.448168, shape=(), dtype=float32)
tf.Tensor(-13.3890085, shape=(), dtype=float32)
tf.Tensor(-13.32981, shape=(), dtype=float32)
tf.Tensor(-13.271203, shape=(), dtype=float32)
tf.Tensor(-13.203066, shape=(), dtype=float32)
tf.Tensor(-13.150055, shape=(), dtype=float32)
tf.Tensor(-13.092884, shape=(), dtype=float32)
tf.Tensor(-13.036259, shape=(), dtype=float32)
tf.Tensor(-12.979974, shape=(), dtype=float32)
tf.Tensor(-12.920391, shape=(), dtype=float32)
tf.Tensor(-12.865106, shape=(), dtype=float32)
tf.Tensor(-12.8104315, shape=(), dtype=float32)
tf.Tensor(-12.75716, shape=(), dtype=float32)
tf.Tensor(-12.703564, shape=(), dtype=float32)
tf.Tensor(-12.650265, shape=(), dtype=float32)
tf.Tensor(-12.597287, shape=(), dtype=float32)
tf.Tensor(-12.542838, shape=(), dtype=float32)
tf.Tensor(-12.490003, shape=(), dtype=float32)
tf.Tensor(-12.438356, shape=(), dtype=float32)
tf.Tensor(-12.387207, shape=(), dtype=float32)
tf.Tensor(-12.336734, shape=(), dtype=float32)
tf.Tensor(-12

...

tf.Tensor(-4.379391, shape=(), dtype=float32)
tf.Tensor(-4.3738117, shape=(), dtype=float32)
tf.Tensor(-4.3682466, shape=(), dtype=float32)
tf.Tensor(-4.3626957, shape=(), dtype=float32)
tf.Tensor(-4.3571596, shape=(), dtype=float32)
tf.Tensor(-4.3516827, shape=(), dtype=float32)
tf.Tensor(-4.346174, shape=(), dtype=float32)
tf.Tensor(-4.3406787, shape=(), dtype=float32)
tf.Tensor(-4.3351984, shape=(), dtype=float32)
tf.Tensor(-4.3297315, shape=(), dtype=float32)
tf.Tensor(-4.324278, shape=(), dtype=float32)
tf.Tensor(-4.3188386, shape=(), dtype=float32)
tf.Tensor(-4.3134127, shape=(), dtype=float32)
tf.Tensor(-4.3080006, shape=(), dtype=float32)
tf.Tensor(-4.3026023, shape=(), dtype=float32)
tf.Tensor(-4.297219, shape=(), dtype=float32)
tf.Tensor(-4.291847, shape=(), dtype=float32)
tf.Tensor(-4.28649, shape=(), dtype=float32)
tf.Tensor(-4.2811446, shape=(), dtype=float32)
tf.Tensor(-4.275812, shape=(), dtype=float32)
tf.Tensor(-4.2704935, shape=(), dtype=float32)
tf.Tensor(-4.265188, 

tf.Tensor(-4.0864058, shape=(), dtype=float32)
tf.Tensor(-4.0815463, shape=(), dtype=float32)
tf.Tensor(-4.076699, shape=(), dtype=float32)
tf.Tensor(-4.0718627, shape=(), dtype=float32)
tf.Tensor(-4.067038, shape=(), dtype=float32)
tf.Tensor(-4.062225, shape=(), dtype=float32)
tf.Tensor(-4.057422, shape=(), dtype=float32)
tf.Tensor(-4.0526333, shape=(), dtype=float32)
tf.Tensor(-4.047819, shape=(), dtype=float32)
tf.Tensor(-4.043051, shape=(), dtype=float32)
tf.Tensor(-4.038294, shape=(), dtype=float32)
tf.Tensor(-4.0335464, shape=(), dtype=float32)
tf.Tensor(-4.0288124, shape=(), dtype=float32)
tf.Tensor(-4.024089, shape=(), dtype=float32)
tf.Tensor(-4.0193768, shape=(), dtype=float32)
tf.Tensor(-4.0146756, shape=(), dtype=float32)
tf.Tensor(-4.009986, shape=(), dtype=float32)
tf.Tensor(-4.005307, shape=(), dtype=float32)
tf.Tensor(-4.000639, shape=(), dtype=float32)
tf.Tensor(-3.9959815, shape=(), dtype=float32)
tf.Tensor(-3.9913347, shape=(), dtype=float32)
tf.Tensor(-3.9866986, sh

tf.Tensor(-3.830829, shape=(), dtype=float32)
tf.Tensor(-3.8265486, shape=(), dtype=float32)
tf.Tensor(-3.822287, shape=(), dtype=float32)
tf.Tensor(-3.8180354, shape=(), dtype=float32)
tf.Tensor(-3.813793, shape=(), dtype=float32)
tf.Tensor(-3.8095598, shape=(), dtype=float32)
tf.Tensor(-3.8053362, shape=(), dtype=float32)
tf.Tensor(-3.8011222, shape=(), dtype=float32)
tf.Tensor(-3.7969174, shape=(), dtype=float32)
tf.Tensor(-3.7922668, shape=(), dtype=float32)
tf.Tensor(-3.7880814, shape=(), dtype=float32)
tf.Tensor(-3.7839038, shape=(), dtype=float32)
tf.Tensor(-3.7797015, shape=(), dtype=float32)
tf.Tensor(-3.7755435, shape=(), dtype=float32)
tf.Tensor(-3.771375, shape=(), dtype=float32)
tf.Tensor(-3.767235, shape=(), dtype=float32)
tf.Tensor(-3.7631054, shape=(), dtype=float32)
tf.Tensor(-3.7589834, shape=(), dtype=float32)
tf.Tensor(-3.754871, shape=(), dtype=float32)
tf.Tensor(-3.750767, shape=(), dtype=float32)
tf.Tensor(-3.746673, shape=(), dtype=float32)
tf.Tensor(-3.7425804,

tf.Tensor(-3.6038206, shape=(), dtype=float32)
tf.Tensor(-3.60004, shape=(), dtype=float32)
tf.Tensor(-3.5962672, shape=(), dtype=float32)
tf.Tensor(-3.592502, shape=(), dtype=float32)
tf.Tensor(-3.5887454, shape=(), dtype=float32)
tf.Tensor(-3.5849152, shape=(), dtype=float32)
tf.Tensor(-3.5818965, shape=(), dtype=float32)
tf.Tensor(-3.578165, shape=(), dtype=float32)
tf.Tensor(-3.57444, shape=(), dtype=float32)
tf.Tensor(-3.5707242, shape=(), dtype=float32)
tf.Tensor(-3.5669992, shape=(), dtype=float32)
tf.Tensor(-3.563295, shape=(), dtype=float32)
tf.Tensor(-3.5595992, shape=(), dtype=float32)
tf.Tensor(-3.555911, shape=(), dtype=float32)
tf.Tensor(-3.5522242, shape=(), dtype=float32)
tf.Tensor(-3.5485532, shape=(), dtype=float32)
tf.Tensor(-3.5448847, shape=(), dtype=float32)
tf.Tensor(-3.541226, shape=(), dtype=float32)
tf.Tensor(-3.537575, shape=(), dtype=float32)
tf.Tensor(-3.5339317, shape=(), dtype=float32)
tf.Tensor(-3.5302968, shape=(), dtype=float32)
tf.Tensor(-3.5266464, s

TypeError: unsupported format string passed to SparseCategoricalAccuracy.__format__

In [ ]:
model

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['accuracy'])

In [ ]:
x=[1,2,3,4,5,6,7,8,9,10]
x=np.asarray(x,dtype=np.float32).reshape((10,1))
y=[1,4,9,16,25,36,49,64,81,100]
y=np.asarray(y,dtype=np.float32).reshape((10,1))
model=model()
for i in range(100):
    model.network_learn(x,y)